In [1]:
#CONVERT MODEL TO TORCH SCRIPT


import torch
# import torchvision.models as models
from models.modelpnnx20x import STBVMM

# net = models.resnet18(pretrained=True)

a_dummy = torch.randn(1, 3, 384, 384)
b_dummy = torch.randn(1, 3, 384, 384)
# c_dummy = torch.randn(1, 3, 384, 384)  # optional during training

# For the amplitude parameter amp
# amp_dummy = torch.tensor(5)  # adjusted to be a scalar value

# Instantiate your model
model = STBVMM()

# Load the weights from the checkpoint
checkpoint = torch.load('ckpt/ckpt_e49.pth.tar')
model.load_state_dict(checkpoint['state_dict'], strict= False)
# Make sure the model is in evaluation mode
# model.eval()
model = model.eval()

# x = torch.rand(1, 3, 224, 224)

# You could try disabling checking when tracing raises error
# mod = torch.jit.trace(net, x, check_trace=False)
mod = torch.jit.trace(model, (a_dummy,b_dummy) )

mod.save("20x/modelpnnx20x.pt")



/opt/homebrew/Caskroom/miniconda/base/envs/afstudeer/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/raoulritter/STB-VMM/models/modelpnnx20x.py:988: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if a.shape != b.shape:
/Users/raoulritter/STB-VMM/models/modelpnnx20x.py:371: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This 

In [13]:
import torch
from models.model_jit_noC import STBVMM

# Set up dummy input data
a_dummy = torch.randn(1, 3, 384, 384)
b_dummy = torch.randn(1, 3, 384, 384)
amp_dummy = torch.tensor(20)

# Instantiate your model
model = STBVMM()

# Load the weights from the checkpoint
checkpoint = torch.load('ckpt/ckpt_e49.pth.tar')
model.load_state_dict(checkpoint['state_dict'], strict= False)

# Make sure the model is in evaluation mode
model = model.eval()


# Trace the model
mod = torch.jit.trace(model, (a_dummy, b_dummy, amp_dummy))

# Save the scripted model
mod.save("20x/modelpnnx20x.pt")


In [21]:
import torch
from PIL import Image
from torchvision import transforms

def main():
    # Load the TorchScript model
    model = torch.jit.load('20x/modelpnnx20x.pt')

    # Load the images from disk
    image_a = Image.open('demo_video/STB-VMM_demo_x20_static_original/frame_000007.png')
    image_b = Image.open('demo_video/STB-VMM_demo_x20_static_original/frame_000008.png')

    # Resize the images if necessary
    desired_size = (384, 384)
    image_a = image_a.resize(desired_size)
    image_b = image_b.resize(desired_size)

    # Convert the images to tensors
    transform = transforms.ToTensor()
    a_tensor = transform(image_a).unsqueeze(0)
    b_tensor = transform(image_b).unsqueeze(0)

    # Set up amp tensor and unsqueeze to match the shape expected by the model
    amp_tensor = torch.tensor([20]).unsqueeze(1).unsqueeze(1).unsqueeze(1)

    # Run inference
    output_tuple = model(a_tensor, b_tensor, amp_tensor)
    output = output_tuple[0]  # Access the desired tensor from the tuple

    # Reshape the tensor (remove batch dimension)
    output = output.squeeze(0)

    # Convert the output tensor to a PIL image
    output_image = transforms.ToPILImage()(output.cpu())

    # Save the combined image
    output_image.save('combined_image.png')


if __name__ == '__main__':
    main()


In [16]:
import torch
from PIL import Image
from torchvision import transforms

def main():
    # Load the TorchScript model
    model = torch.jit.load('20x/modelpnnx20x.pt')

    # Load the images from disk
    image_a = Image.open('demo_video/STB-VMM_demo_x20_static_original/frame_000007.png')
    image_b = Image.open('demo_video/STB-VMM_demo_x20_static_original/frame_000008.png')

    # Resize the images if necessary
    desired_size = (384, 384)
    image_a = image_a.resize(desired_size)
    image_b = image_b.resize(desired_size)

    # Convert the images to tensors
    transform = transforms.ToTensor()
    a_tensor = transform(image_a).unsqueeze(0)
    b_tensor = transform(image_b).unsqueeze(0)

    # Set up amp tensor
    amp_tensor = torch.tensor([20])  # Replace 20 with your desired amplification factor

    # Run inference
    output_tuple = model(a_tensor, b_tensor, amp_tensor)
    output = output_tuple[0]  # Access the desired tensor from the tuple

    # Reshape the tensor (remove batch dimension)
    output = output.squeeze(0)

    # Convert the output tensor to a PIL image
    output_image = transforms.ToPILImage()(output.cpu())

    # Save the combined image
    output_image.save('combined_image.png')


if __name__ == '__main__':
    main()


In [24]:
#INFERENCING WITH TORCHSCRIPT IN ORDER TO GET ONE OUTPUT IMAGE

import torch
from PIL import Image
from torchvision import transforms

def main():
    # Load the TorchScript model
    model = torch.jit.load('20x/modelpnnx20x.pt')

    # Load the images from disk
    image_a = Image.open('demo_video/STB-VMM_demo_x20_static_original/frame_000007.png')
    image_b = Image.open('demo_video/STB-VMM_demo_x20_static_original/frame_000008.png')

    # Resize the images if necessary
    desired_size = (384, 384)
    image_a = image_a.resize(desired_size)
    image_b = image_b.resize(desired_size)

    # Convert the images to tensors
    transform = transforms.ToTensor()
    a_tensor = transform(image_a).unsqueeze(0)
    b_tensor = transform(image_b).unsqueeze(0)

    # Run inference
    output_tuple = model(a_tensor, b_tensor)
    output = output_tuple[0]  # Access the desired tensor from the tuple

    # Reshape the tensor (remove batch dimension)
    output = output.squeeze(0)

    # Convert the output tensor to a PIL image
    output_image = transforms.ToPILImage()(output.cpu())

    # Save the combined image
    output_image.save('combined_image.png')


if __name__ == '__main__':
    main()


RuntimeError: version_ <= kMaxSupportedFileFormatVersionINTERNAL ASSERT FAILED at "/Users/distiller/project/pytorch/caffe2/serialize/inline_container.cc":147, please report a bug to PyTorch. Attempted to read a PyTorch file with version 10, but the maximum supported version for reading is 9. Your PyTorch installation may be too old.

In [23]:
import torch
from PIL import Image
from torchvision import transforms
import os

def main():
    # Load the TorchScript model
    model = torch.jit.load('20x/modelpnnx20x.pt')

    # Directory path of input frames
    frames_dir = 'demo_video/STB-VMM_demo_x20_static_original'

    # Output directory path to save generated frames
    output_dir = 'output_frames'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Resize the images if necessary
    desired_size = (384, 384)

    # Get the list of frames in the directory
    frame_files = sorted(os.listdir(frames_dir))

    # Process the frames in pairs and generate output images
    num_frames = len(frame_files)
    num_pairs = min(num_frames // 2, 150)

    for i in range(num_pairs):
        # Load the pair of frames
        frame_a_path = os.path.join(frames_dir, frame_files[2 * i])
        frame_b_path = os.path.join(frames_dir, frame_files[2 * i + 1])

        image_a = Image.open(frame_a_path)
        image_b = Image.open(frame_b_path)

        # Resize the images
        image_a = image_a.resize(desired_size)
        image_b = image_b.resize(desired_size)

        # Convert the images to tensors
        transform = transforms.ToTensor()
        a_tensor = transform(image_a).unsqueeze(0)
        b_tensor = transform(image_b).unsqueeze(0)

        # Set up amp tensor and unsqueeze to match the shape expected by the model
        amp_tensor = torch.tensor([20]).unsqueeze(1).unsqueeze(1).unsqueeze(1)

        # Run inference
        output_tuple = model(a_tensor, b_tensor, amp_tensor)
        output = output_tuple[0]  # Access the desired tensor from the tuple

        # Reshape the tensor (remove batch dimension)
        output = output.squeeze(0)

        # Convert the output tensor to a PIL image
        output_image = transforms.ToPILImage()(output.cpu())

        # Save the output image
        output_path = os.path.join(output_dir, f'output_{i + 1:03d}.png')
        output_image.save(output_path)

        if (i + 1) % 10 == 0:
            print(f'Processed {i + 1} pairs out of {num_pairs}')

    print('Processing completed.')


if __name__ == '__main__':
    main()


Processed 10 pairs out of 150
Processed 20 pairs out of 150
Processed 30 pairs out of 150
Processed 40 pairs out of 150
Processed 50 pairs out of 150
Processed 60 pairs out of 150
Processed 70 pairs out of 150
Processed 80 pairs out of 150
Processed 90 pairs out of 150
Processed 100 pairs out of 150
Processed 110 pairs out of 150
Processed 120 pairs out of 150
Processed 130 pairs out of 150
Processed 140 pairs out of 150
Processed 150 pairs out of 150
Processing completed.


In [6]:
#INFERENCE WITH ALL FRAMES FROM DIRECTORY

import torch
from PIL import Image
from torchvision import transforms
import os

def main():
    # Load the TorchScript model
    model = torch.jit.load('20x/modelpnnx20x.pt')

    # Directory path of input frames
    frames_dir = 'demo_video/STB-VMM_demo_x20_static_original'

    # Output directory path to save generated frames
    output_dir = 'output_frames'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Resize the images if necessary
    desired_size = (384, 384)

    # Get the list of frames in the directory
    frame_files = sorted(os.listdir(frames_dir))

    # Process the frames in pairs and generate output images
    num_frames = len(frame_files)
    num_pairs = min(num_frames // 2, 150)

    for i in range(num_pairs):
        # Load the pair of frames
        frame_a_path = os.path.join(frames_dir, frame_files[2 * i])
        frame_b_path = os.path.join(frames_dir, frame_files[2 * i + 1])

        image_a = Image.open(frame_a_path)
        image_b = Image.open(frame_b_path)

        # Resize the images
        image_a = image_a.resize(desired_size)
        image_b = image_b.resize(desired_size)

        # Convert the images to tensors
        transform = transforms.ToTensor()
        a_tensor = transform(image_a).unsqueeze(0)
        b_tensor = transform(image_b).unsqueeze(0)

        # Run inference
        output_tuple = model(a_tensor, b_tensor)
        output = output_tuple[0]  # Access the desired tensor from the tuple

        # Reshape the tensor (remove batch dimension)
        output = output.squeeze(0)

        # Convert the output tensor to a PIL image
        output_image = transforms.ToPILImage()(output.cpu())

        # Save the output image
        output_path = os.path.join(output_dir, f'output_{i + 1:03d}.png')
        output_image.save(output_path)

        if (i + 1) % 10 == 0:
            print(f'Processed {i + 1} pairs out of {num_pairs}')

    print('Processing completed.')


if __name__ == '__main__':
    main()


Processed 10 pairs out of 150
Processed 20 pairs out of 150
Processed 30 pairs out of 150
Processed 40 pairs out of 150
Processed 50 pairs out of 150
Processed 60 pairs out of 150
Processed 70 pairs out of 150
Processed 80 pairs out of 150
Processed 90 pairs out of 150
Processed 100 pairs out of 150
Processed 110 pairs out of 150
Processed 120 pairs out of 150
Processed 130 pairs out of 150
Processed 140 pairs out of 150
Processed 150 pairs out of 150
Processing completed.


In [10]:
import os
import numpy as np
import torch
from PIL import Image
from torchvision import transforms

def main():
    # Load the TorchScript model
    model = torch.jit.load('20x/modelpnnx20x.pt')

    # Check saving directory
    save_dir = 'demo'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    print(save_dir)

    # Load the images from disk
    frame_files = sorted(os.listdir('demo_video/STB-VMM_demo_x20_static_original'))

    # Resize the images if necessary
    desired_size = (384, 384)

    # Process the frames and generate output images
    num_frames = len(frame_files)
    num_pairs = min(num_frames // 2, 150)

    for i in range(num_pairs):
        # Load the pair of frames
        frame_a_path = os.path.join('demo_video/STB-VMM_demo_x20_static_original', frame_files[2 * i])
        frame_b_path = os.path.join('demo_video/STB-VMM_demo_x20_static_original', frame_files[2 * i + 1])

        image_a = Image.open(frame_a_path)
        image_b = Image.open(frame_b_path)

        # Resize the images
        image_a = image_a.resize(desired_size)
        image_b = image_b.resize(desired_size)

        # Convert the images to tensors
        transform = transforms.ToTensor()
        a_tensor = transform(image_a).unsqueeze(0)
        b_tensor = transform(image_b).unsqueeze(0)

        # Run inference
        output_tuple = model(a_tensor, b_tensor)
        output = output_tuple[0][-3:, :, :]  # Take the last 3 channels

        # Convert the output tensor to a PIL image
        output_image = transforms.ToPILImage()(output.cpu())

        # Save the output image
        output_file = f"output_{i}.png"
        output_path = os.path.join(save_dir, output_file)
        output_image.save(output_path)

        if (i+1) == num_pairs:
            break

if __name__ == '__main__':
    main()


demo


ValueError: pic should be 2/3 dimensional. Got 4 dimensions.

In [7]:
import onnx

# Load the ONNX model
model = onnx.load("20x/modelpnnx20x.pnnx.onnx")

# Add the opset version
model.opset_import.add()
model.opset_import[0].version = 11
model.ir_version = 7


# Check the model
print(model)

# Save the modified model
onnx.save(model, "model_with_opset.onnx")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [1]:
import torch
import onnxruntime
from torchvision import transforms
from PIL import Image

def main():
    # Load the ONNX model
    onnx_model_path = 'model_with_opset.onnx'

    ort_session = onnxruntime.InferenceSession(onnx_model_path)

    # Load the images from disk
    image_a = Image.open('demo_video/STB-VMM_demo_x20_static_original/frame_000007.png')
    image_b = Image.open('demo_video/STB-VMM_demo_x20_static_original/frame_000008.png')

    # Resize the images if necessary
    desired_size = (384, 384)
    image_a = image_a.resize(desired_size)
    image_b = image_b.resize(desired_size)

    # Convert the images to tensors
    transform = transforms.ToTensor()
    a_tensor = transform(image_a).unsqueeze(0)
    b_tensor = transform(image_b).unsqueeze(0)

    # Convert the input tensors to NumPy arrays
    a_np = a_tensor.numpy()
    b_np = b_tensor.numpy()

    # Run inference using ONNX Runtime
    input_names = ort_session.get_inputs()[0].name
    output_names = [output.name for output in ort_session.get_outputs()]
    output = ort_session.run(output_names, {input_names: (a_np, b_np)})

    # Convert the output tensor to a PIL image
    output_tensor = torch.from_numpy(output[0])
    output_image = transforms.ToPILImage()(output_tensor.cpu())

    # Save the combined image
    output_image.save('combined_image.png')

if __name__ == '__main__':
    main()


Fail: [ONNXRuntimeError] : 1 : FAIL : Load model from model_with_opset.onnx failed:Invalid tensor data type 0.

## Everything below can be ignored as this is what was used during prevous testing

In [21]:
import torch
from models.model2H5A import STBVMM

# Define dummy inputs of the appropriate sizes for your model

a_dummy = torch.randn(1, 3, 384, 384)
b_dummy = torch.randn(1, 3, 384, 384)
# c_dummy = torch.randn(1, 3, 384, 384)  # optional during training

# For the amplitude parameter amp
amp_dummy = torch.randn(1)  # adjusted to be a scalar value

# Instantiate your model
model = STBVMM()

# Load the weights from the checkpoint
checkpoint = torch.load('ckpt/ckpt_e49.pth.tar')
model.load_state_dict(checkpoint['state_dict'], strict= False)
# Make sure the model is in evaluation mode
model.eval()

# Export the model to ONNX
torch.onnx.export(model, (a_dummy,b_dummy, amp_dummy), 'modelforpnnx.onnx')


TypeError: forward() takes from 2 to 3 positional arguments but 4 were given

In [ ]:
import torch
from models.model import STBVMM  

# Define dummy inputs of the appropriate sizes for your model

a_dummy = torch.randn(1, 3, 384,384)
b_dummy = torch.randn(1, 3, 384, 384)
# c_dummy = torch.randn(1, 3, 384, 384)  # optional during training

# For the amplitude parameter amp
amp_dummy = torch.randn(5)  # adjusted to be a scalar value

# Instantiate your model
model = STBVMM()

# Load the weights from the checkpoint
checkpoint = torch.load('ckpt/ckpt_e49.pth.tar')
model.load_state_dict(checkpoint['state_dict'], strict= False)
# Make sure the model is in evaluation mode
model.eval()
# model.pt¸

# # Export the model to ONNX
# torch.onnx.export(model, 
#                   (a_dummy, amp_dummy), 
#                   opset_version=10,
#                   'model.onnx')

# torch.onnx.export(model,               # model being run
#                   example_input,       # model input (or a tuple for multiple inputs)
#                   "model.onnx",        # where to save the model (can be a file or file-like object)
#                   export_params=True,  # store the trained parameter weights inside the model file
# Export the model to ONNX
torch.onnx.export(model, 
                  (a_dummy,b_dummy, amp_dummy), 
                  'model.onnx',
                  opset_version=11)



/Users/raoulritter/STB-VMM/models/model.py:988: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if a.shape != b.shape:
/Users/raoulritter/STB-VMM/models/model.py:153: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
/Users/raoulritter/STB-VMM/models/model.py:371: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record

KeyboardInterrupt: 